In [ ]:
import tensorflow as tf
import pandas as pd
import librosa
import numpy as np
from pydub import AudioSegment
from io import BytesIO

In [43]:
tf.__version__

'2.18.0'

In [44]:
# load model
model = tf.keras.models.load_model('model/model.keras')

# load class names
class_names = pd.read_csv('model/class_names.csv')

In [45]:
class_names

,class_name,class_code
0,Aek Kapus,0
1,Akai Bipamare,1
2,Ampar-Ampar Pisang,2
3,Anak Daro,3
4,Anak Kambing Saya,4
...,...,...
98,Tumpi Wayu,98
99,Tutu Koda,99
100,Waktu Hujan Sore-Sore,100
101,Yamko Rambe Yamko,101


In [46]:
def convert_to_wav(audio_path):
    # check if already wav
    if audio_path.endswith('.wav'):
        return audio_path
    
    # load audio
    audio = AudioSegment.from_file(audio_path)
    
    # convert to wav
    wav_audio = BytesIO()
    
    audio.export(wav_audio, format='wav')
    
    wav_audio.seek(0)
    
    return wav_audio

def mfcc_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T,axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs_processed

def predict_song(model, file_path):
    wav = convert_to_wav(file_path)
    features = mfcc_features(wav)
    
    if features is None:
        return None
    
    features = features.reshape(1, -1)
    pred = model.predict(features)
    
    pred_class = np.argmax(pred)
    pred_prob = pred[0][pred_class]
    
    return pred_class, pred_prob

In [47]:
# read folder test to predict
import os

folder = 'test'

for file in os.listdir(folder):
    print(f'Predicting {file}')
    
    predict_title, pred_prob = predict_song(model, os.path.join(folder, file))
    print(f'Predicted class: {class_names.iloc[predict_title, 0]} with probability {pred_prob:.2f}\n')

Predicting SERU_JUGA_Ikutan_Tari_kreasi_manuk_dadali_[_YouConvert.net_].mp3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted class: Butet with probability 0.21

Predicting KARAOKE_MANUK_DADALI_Lagu_Daerah_Jawa_Barat_[_YouConvert.net_].mp3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: Manuk Dadali with probability 0.98

Predicting manuk_dadali_manual1.m4a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: Tumpi Wayu with probability 0.46

Predicting manuk_dadali_manual2.m4a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted class: Borero with probability 0.99

